# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex, 
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)

In [3]:
# additional CSS to consider. 
# TODO: Current into each exported file. 
#       This should be set once into the webpages directly.
#       With only the class/id definitions in the .md files.

debug_html = """
<style>
#wrap{ overflow:auto; }
#fig1{ background:yellow; width:100%; float:left; padding:5px;  }
#fig2{ background:red; width:50%; float:left; clear:left; padding:5px;  }
#fig3{ background:green; width:50%; float:left; padding:5px;   }
.macros{ background:yellow; visibility:visible;}
h1 {margin: 0 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
""" 
html = """
<style>
#wrap{ overflow:auto; }
#fig1{ width:100%; float:left; padding: 5px;  }
#fig2{ width:50%; float:left; clear:left; padding: 5px;  }
#fig3{ width:50%; float:left; padding: 5px;  }
.macros{ visibility:hidden; height:0px; }
h1 {margin: 0em 0 0 0;}
mark {background-color:#fff3b6;}
img {object-fit:contain; max-height:250px; display:inline-block; text-align: center;}
</style>
"""

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [4]:
# get list from MPIA website
# it automatically filters identified non-scientists :func:`mpia.filter_non_scientists`
mpia_authors = mpia.get_mpia_mitarbeiter_list()
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

# select only papers with matching author names and highlight authors
hl_list = [k[0] for k in mpia_authors]

candidates = []
for paperk in new_papers:
    hl_authors = highlight_authors_in_list(paperk['authors'], hl_list)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

Arxiv has 50 new papers today
          19 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates[:-1]):
    paper_id = paper['identifier'].lower().replace('arxiv:', '')
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = get_markdown_badge(paper_id) + " _" + paper['comments'] + "_"
        doc.highlight_authors_in_list(hl_list)

        full_md = doc.generate_markdown_text()
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/18 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2208.08450


extracting tarball to tmp_2208.08450...

 done.
Retrieving document from  https://arxiv.org/e-print/2208.08473


extracting tarball to tmp_2208.08473...

...

 done.


/opt/hostedtoolcache/Python/3.9.13/x64/lib/python3.9/site-packages/arxiv_on_deck_2/latex.py:118: LatexWarning: attempting recovering figure pictures/no_lithium_line_Li_comparison_GIRAFFE.png
  warnings.warn(LatexWarning(f'attempting recovering figure {image}'))
/tmp/ipykernel_2208/1244451042.py:32: LatexWarning: 2208.08874 did not run properly
Could not find figure pictures/no_lithium_line_Li_comparison_GIRAFFE.png
  warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +


Retrieving document from  https://arxiv.org/e-print/2208.08908


extracting tarball to tmp_2208.08908...

 done.


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08872-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08872) | **The Gaia-ESO Survey: Chemical evolution of Mg and Al in the Milky Way  with Machine-Learning**  |
|| M. Ambrosch, et al. -- incl., <mark>M. Bergemann</mark> |
|*Appeared on*| *2022-08-19*|
|*Comments*| *Submitted to A&A*|
|**Abstract**| We aim to prepare the machine-learning ground for the next generation of spectroscopic surveys, such as 4MOST and WEAVE. Our goal is to show that convolutional neural networks can predict accurate stellar labels from relevant spectral features in a physically meaningful way. We built a neural network and trained it on GIRAFFE spectra with associated stellar labels from the sixth internal Gaia-ESO data release. Our neural network predicts the atmospheric parameters Teff and log(g) as well as the chemical abundances [Mg/Fe], [Al/Fe], and [Fe/H] for 30115 stellar spectra. The scatter of predictions from eight slightly different network models shows a high internal precision of the network results: 24 K for Teff, 0.03 for log(g), 0.02 dex for [Mg/Fe], 0.03 dex for [Al/Fe], and 0.02 dex for [Fe/H]. The network gradients reveal that the network is inferring the labels in a physically meaningful way from spectral features. Validation with benchmark stars and several scientific applications confirm that our network predictions are accurate for individual stars and recover the properties of different stellar populations in the Milky Way galaxy. Such a study provides very good insights into the application of machine-learning for the spectral analysis of large-scale spectroscopic surveys, such as WEAVE and 4MIDABLE-LR and -HR (4MOST Milky Way disk and bulge low- and high-resolution). The community will have to put a substantial effort into building proactive training sets for machine-learning methods to minimize the possible systematics. |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08450-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08450) | **Long-term photometric monitoring and spectroscopy of the white dwarf  pulsar AR Scorpii**  |
|| Ingrid Pelisoli, et al. -- incl., <mark>A. J. Brown</mark>, <mark>M. J. Green</mark> |
|*Appeared on*| *2022-08-19*|
|*Comments*| *15 pages, 18 figures. Accepted for publication in MNRAS*|
|**Abstract**| AR Scorpii (AR Sco) is the only radio-pulsing white dwarf known to date. It shows a broad-band spectrum extending from radio to X-rays whose luminosity cannot be explained by thermal emission from the system components alone, and is instead explained through synchrotron emission powered by the spin-down of the white dwarf. We analysed NTT/ULTRACAM, TNT/ULTRASPEC, and GTC/HiPERCAM high-speed photometric data for AR Sco spanning almost seven years and obtained a precise estimate of the spin frequency derivative, now confirmed with 50-sigma significance. Using archival photometry, we show that the spin down rate of P/Pdot = 5.6e6 years has remained constant since 2005. As well as employing the method of pulse-arrival time fitting used for previous estimates, we also found a consistent value via traditional Fourier analysis for the first time. In addition, we obtained optical time-resolved spectra with WHT/ISIS and VLT/X-shooter. We performed modulated Doppler tomography for the first time for the system, finding evidence of emission modulated on the orbital period. We have also estimated the projected rotational velocity of the M-dwarf as a function of orbital period and found that it must be close to Roche lobe filling. Our findings provide further constraints for modelling this unique system. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08473-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08473) | **The Identification of a Dusty Grand Design Spiral Galaxy at $z=3.06$  with JWST and ALMA**  |
|| Yunjing Wu, et al. -- incl., <mark>Shiwu Zhang</mark> |
|*Appeared on*| *2022-08-19*|
|*Comments*| *To be submitted to ApJL. 10 pages, four figures in the main text, and two figures in the appendix. Comments are very welcome*|
|**Abstract**| Spiral arms serve crucial purposes in star formation and galaxy evolution. In this paper, we report the identification of "A2744-DSG-$z3$", a dusty, grand-design-like spiral galaxy at $z=3.059$ using the James Webb Space Telescope (JWST) NIRISS imaging and grism spectroscopy. A2744-DSG-$z3$ was discovered as a gravitationally lensed sub-millimeter galaxy with ALMA. This is the most distant grand-design stellar spiral structure seen thus far, consistent with cosmological simulations which suggest $z\approx3$ as the epoch when grand-design spirals emerge. Thanks to the gravitational lensing and excellent spatial resolution of JWST, the spiral arms are resolved with a spatial resolution of $\approx290$\,pc. Based on SED fitting, after correcting for lensing, the spiral galaxy has a de-lensed star formation rate of $85\pm30 \ M_{\odot}$ yr$^{-1}$, and a stellar mass of $\approx10^{10.6}\ M_{\odot}$, indicating that A2744-DSG-$z3$ is a main-sequence galaxy. After fitting the spiral arms, we find a stellar effective radii ($R_{e, \rm{star}}$) is $7.3 \pm 0.8$ kpc. Comparing to ALMA, we find that the effective radii ratio between dust and stars is $\approx0.2$, similar to that of massive SFGs at $z\sim2$, indicating a compact dusty core in A2744-DSG-$z3$. Moreover, this galaxy appears to be living in a group environment: including A2744-DSG-$z3$, at least three galaxies at $z=3.05 - 3.06$ spectroscopically confirmed by JWST/NIRISS and ALMA, and residing within a lensing-corrected, projected scale of $\approx 70$ kpc. This, along with the asymmetric brightness profile, further suggests that the spiral arms may be triggered by minor merger events at $z\gtrsim3$. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

...


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08694-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08694) | **The integrated radio spectrum of G2.4$+$1.4**  |
|| <mark>D. A. Green</mark> |
|*Appeared on*| *2022-08-19*|
|*Comments*| *5 pages*|
|**Abstract**| The Galactic source G2.4$+$1.4 is an optical and radio nebula containing an extreme Wolf--Rayet star. At one time this source was regarded as a supernova remnant, because of its apparent non-thermal radio spectrum, although this was based on limited observations. Subsequent observations instead supported a flat, optically thin thermal radio spectrum for G2.4$+$1.4, and it was identified as a photoionized, mass-loss bubble, not a supernova remnant. Recently, however, it has been claimed that this source has a non-thermal integrated radio spectrum. I discuss the integrated radio flux densities available for G2.4$+$1.4 from a variety of surveys, and show that it has a flat spectrum at gigahertz frequencies (with a spectral index $\alpha$ of $0.02 \pm 0.08$, where flux density $S$ scales with frequency $\nu$ as $S \propto \nu^{-\alpha}$). |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08712-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08712) | **Faraday Rotation Measure Variations of Repeating Fast Radio Burst  Sources**  |
|| Yuan-Pei Yang, Siyao Xu, <mark>Bing Zhang</mark> |
|*Appeared on*| *2022-08-19*|
|*Comments*| *15 pages, 6 figures. Submitted to MNRAS. Comments welcome!*|
|**Abstract**| Recently, some fast radio burst (FRB) repeaters were reported to exhibit complex, diverse variations of Faraday rotation measures (RMs), which implies that they are surrounded by an inhomogeneous, dynamically evolving, magnetized environment. We systematically investigate some possible astrophysical processes that may cause RM variations of an FRB repeater. The processes include (1) a supernova remnant (SNR) with a fluctuating medium; (2) a binary system with stellar winds from a massive/giant star companion or stellar flares from a low-mass star companion; (3) a pair plasma medium from a neutron star (including pulsar winds, pulsar wind nebulae and magnetar flares); (4) outflows from a massive black hole. For the SNR scenario, a large relative RM variation during a few years requires that the SNR is young with a thin and local anisotropic shell, or the size of dens gas clouds in interstellar/circumstellar medium around the SNR is extremely small. If the RM variation is caused by the companion medium in a binary system, it is more likely from stellar winds of a massive/giant star companion. The RM variation contributed by stellar flares from a low-mass star is disfavored, because this scenario predicts an extremely large relative RM variation during a short period of time. The scenarios invoking a pair plasma from a neutron star can be ruled out due to their extremely low RM contributions. Outflows from a massive black hole could provide a large RM variation if the FRB source is in the vicinity of the black hole. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08849-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08849) | **Bridging the gap between Monte Carlo simulations and measurements of the  LISA Pathfinder test-mass charging for LISA**  |
|| Catia Grimani, et al. -- incl., <mark>Mattia Villani</mark> |
|*Appeared on*| *2022-08-19*|
|*Comments*| *11 pages, 9 figures*|
|**Abstract**| Cubic gold-platinum free-falling test masses (TMs) constitute the mirrors of future LISA and LISA-like interferometers for low-frequency gravitational wave detection in space. High-energy particles of Galactic and solar origin charge the TMs and thus induce spurious electrostatic and magnetic forces that limit the sensitivity of these interferometers. Prelaunch Monte Carlo simulations of the TM charging were carried out for the LISA Pathfinder (LPF) mission, that was planned to test the LISA instrumentation. Measurements and simulations were compared during the mission operations. The measured net TM charging agreed with simulation estimates, while the charging noise was three to four times higher. We aim to bridge the gap between LPF TM charging noise simulations and observations. New Monte Carlo simulations of the LPF TM charging due to both Galactic and solar particles were carried out with the FLUKA/LEI toolkit. This allowed propagating low-energy electrons down to a few electronvolt. These improved FLUKA/LEI simulations agree with observations gathered during the mission operations within statistical and Monte Carlo errors. The charging noise induced by Galactic cosmic rays is about one thousand charges per second. This value increases to tens of thousands charges per second during solar energetic particle events. Similar results are expected for the LISA TM charging. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08908-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08908) | **L-band nulling interferometry at the VLTI with Asgard/Hi-5: status and  plans**  |
|| Denis Defrère, et al. -- incl., <mark>Simon Gross</mark>, <mark>Helene Rousseau</mark> |
|*Appeared on*| *2022-08-19*|
|*Comments*| *16 pages, 9 figures, SPIE 2022 "Astronomical Telescopes and Instrumentation" manuscript 12183-16*|
|**Abstract**| Hi-5 is the L'-band (3.5-4.0 $\mu$m) high-contrast imager of Asgard, an instrument suite in preparation for the visitor focus of the VLTI. The system is optimized for high-contrast and high-sensitivity imaging within the diffraction limit of a single UT/AT telescope. It is designed as a double-Bracewell nulling instrument producing spectrally-dispersed (R=20, 400, or 2000) complementary nulling outputs and simultaneous photometric outputs for self-calibration purposes. In this paper, we present an update of the project with a particular focus on the overall architecture, opto-mechanical design of the warm and cold optics, injection system, and development of the photonic beam combiner. The key science projects are to survey (i) nearby young planetary systems near the snow line, where most giant planets are expected to be formed, and (ii) nearby main sequence stars near the habitable zone where exozodiacal dust that may hinder the detection of Earth-like planets. We present an update of the expected instrumental performance based on full end-to-end simulations using the new GRAVITY+ specifications of the VLTI and the latest planet formation models. |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-arXiv:2208.08874-b31b1b.svg)](https://arxiv.org/abs/arXiv:2208.08874) | **The Gaia-ESO Survey: Preparing the ground for 4MOST & WEAVE galactic  surveys -- Chemical Evolution of Lithium with Machine-Learning**  |
|| S. Nepal, et al. -- incl., <mark>M. Bergemann</mark> |
|*Appeared on*| *2022-08-19*|
|*Comments*| *Submitted to A&A*|
|**Abstract**| Originating from several sources (Big Bang, stars, cosmic rays) and being strongly depleted during stellar lifetime, the lithium element (Li) is of great interest as its chemical evolution in the Milky Way is not yet well understood. To help constrain stellar and galactic chemical evolution models, numerous and precise Li abundances are necessary for a large range of evolutionary stages, metallicities, and Galactic volume. In the age of industrial parametrization, spectroscopic surveys such as APOGEE, GALAH, RAVE, and LAMOST have used data-driven methods to rapidly and precisely infer stellar labels (atmospheric parameters and abundances). To prepare grounds for future spectroscopic surveys like 4MOST and WEAVE, we aim to apply machine-learning techniques for Li study/measurement. We train a Convolution Neural-Network (CNN) coupling Gaia-ESO Survey iDR6 stellar labels ($\mathrm{\textit{T}_{\text{eff}}}$, log(g), [Fe/H] and A(Li)) and GIRAFFE HR15N spectra, to infer the atmospheric parameters and Li abundances for $\sim$40000 stars. We show that the CNN properly learns the physics of the stellar labels, from relevant spectral features, over a large range of evolutionary stages and stellar parameters. The Li feature at 6707.8 A is successfully singled out by our CNN, among the thousands of lines in the GIRAFFE HR15N setup. Rare objects like Li-rich giants are found in our sample. Such performances are achieved thanks to a meticulously built high-quality and homogeneous training sample. The CNN approach is very well adapted for the next generations of spectroscopic surveys aiming at studying (among other elements) Li, such as the 4MIDABLE-LR/HR (4MOST Milky Way disk and bulge low- and high-resolution) surveys. In this context, the caveats of the machine-learning applications should be properly investigated along with realistic label uncertainties and upper limits for abundances. |
|<p style="color:red"> **ERROR** </p>| <p style="color:red">latex error Could not find figure pictures/no_lithium_line_Li_comparison_GIRAFFE.png</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

exported in  _build/html/2208.08872.md
    + _build/html/tmp_2208.08872/./kiel_GES.png
    + _build/html/tmp_2208.08872/./density_MgAl_Fe.png
    + _build/html/tmp_2208.08872/./disks.png
    + _build/html/tmp_2208.08872/./disks_obs.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$</div>

<div class="macros" style="visibility:hidden;">
$\newcommand{$\ensuremath$}{}$
$\newcommand{$\xspace$}{}$
$\newcommand{$\object$}[1]{\texttt{#1}}$
$\newcommand{$\farcs$}{{.}''}$
$\newcommand{$\farcm$}{{.}'}$
$\newcommand{$\arcsec$}{''}$
$\newcommand{$\arcmin$}{'}$
$\newcommand{$\ion$}[2]{#1#2}$
$\newcommand{$\textsc$}[1]{\textrm{#1}}$
$\newcommand{$\hl$}[1]{\textrm{#1}}$</div>



<div id="title">

# Based on observations made with the ESO/VLT, at Paranal Observatory, under program 188.B-3002 (The Gaia-ESO Public Spectroscopic Survey, PIs G. Gilmore and S. Randich). Also based on observations under programs 171.0237 and 073.0234

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2208.08872-b31b1b.svg)](https://arxiv.org/abs/2208.08872) _Submitted to A&A_

</div>
<div id="authors">

M. Ambrosch, et al. -- incl., <mark><mark>M. Bergemann</mark></mark>

</div>
<div id="abstract">

**Abstract:** To take full advantage of upcoming large-scale spectroscopic surveys, it will be necessary to parameterize millions of stellar spectra in an efficient way. Machine--learning methods, and especially convolutional neural networks, will be one of the main tools for achieving this task.We aim to prepare the machine--learning ground for the next generation of spectroscopic surveys, such as 4MOST and WEAVE. Our goal is to show that convolutional neural networks can predict accurate stellar labels from relevant spectral features in a physically meaningful way. The predicted labels can be used to investigate properties of the Milky Way galaxy.We built a neural network and trained it on GIRAFFE spectra with associated stellar labels from the sixth internal Gaia-ESO data release. Our network architecture contains several convolutional layers that allow the network to identify absorption features in the input spectra. Internal uncertainty is estimated from multiple network models. We used\textit{t-distributed Stochastic Neighbor Embedding}to remove bad spectra from our training sample.Our neural network predicts the atmospheric parameters\textit{T}\textsubscript{eff}and log(\textit{g}) as well as the chemical abundances[Mg/Fe],[Al/Fe], and[Fe/H]for 30 115 stellar spectra. The internal uncertainty is 24 K for\textit{T}\textsubscript{eff}, 0.03 for log(\textit{g}), 0.02 dex for[Mg/Fe], 0.03 dex for[Al/Fe], and 0.02 dex for[Fe/H]. The network gradients reveal that the network is inferring the labels in a physically meaningful way from spectral features. We validated our methodology using benchmark stars and  recovered the properties of different stellar populations in the Milky Way galaxy.Such a study provides very good insights into the application of machine--learning for the analysis of large-scale spectroscopic surveys, such as WEAVE and 4MIDABLE-LR and -HR (4MOST Milky Way disk and bulge low- and high-resolution). The community will have to put a substantial effort in building proactive training sets for machine--learning methods to minimize the possible systematics.

</div>

<div id="div_fig1">

<img src="tmp_2208.08872/./kiel_GES.png" alt="Fig1" width="100%"/>

**Figure 1. -** Kiel diagram containing the stars that will be used to train and test our neural network. The color-coding indicates the metallicity gradient in the giant branch stars. (*fig:kiel GES*)

</div>
<div id="div_fig2">

<img src="tmp_2208.08872/./density_MgAl_Fe.png" alt="Fig2" width="100%"/>

**Figure 2. -** Density plots of [Mg/Fe] vs. [Fe/H](top panel) and [Al/Fe] vs. [Fe/H](bottom panel) for the 14 696  stars in the training and test sets. Brighter colors indicate a higher density of data points. (*fig:MgAl_Fe_hist2d*)

</div>
<div id="div_fig3">

<img src="tmp_2208.08872/./disks.png" alt="Fig10.1" width="50%"/><img src="tmp_2208.08872/./disks_obs.png" alt="Fig10.2" width="50%"/>

**Figure 10. -** Top panel: Density map of the [Mg/Fe] vs. [Fe/H] distribution of our CNN results for the training+test sample. Brighter colors indicate a higher density of data points. Thin and thick disk populations found by the HDBSCAN algorithm are shown on the right. The two populations correspond to the two separate dense regions on the left panel. Bottom: same plot but for the observed sample composed of 15 419 stars with 20 $\le$ S/N $\le$ 30. (*fig:disks*)

</div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")
[ print('\t', k) for k in res ];

19  publications in the last 7 days.
	 _build/html/2208.08872.md
	 _build/html/2204.06393.md
	 _build/html/2204.03335.md
	 _build/html/2204.03253.md
	 _build/html/2204.02998.md
	 _build/html/2204.02109.md
	 _build/html/2204.02017.md
	 _build/html/2204.01824.md
	 _build/html/2204.01758.md
	 _build/html/2204.01245.md
	 _build/html/2204.00793.md
	 _build/html/2204.00342.md
	 _build/html/2203.16959.md
	 _build/html/2203.16856.md
	 _build/html/2203.16735.md
	 _build/html/2203.16734.md
	 _build/html/2203.16504.md
	 _build/html/2203.15822.md
	 _build/html/2203.15811.md


In [11]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""<section class="carousel" aria-label="Gallery">""",
                """  <ol class="carousel__viewport">""",
    ]
    for k in range(1, npub + 1):
        prev_ = k - 1
        next_ = k + 1
        if prev_ <= 0:
            prev_ = npub
        if next_ > npub:
            next_ = 1
        text  = f"""    <li id="carousel__slide{k}" tabindex="0" class="carousel__slide">\n"""
        text += f"""       <div class="carousel__snapper">\n"""
        text += f"""         <a href="#carousel__slide{prev_}" class="carousel__prev">Go to previous slide</a>\n"""
        text += f"""         <a href="#carousel__slide{next_}" class="carousel__next">Go to next slide</a>\n"""
        text += f"""         <div id="slide{k}_content" class="md_view" >Content {k}</div>\n"""
        text += f"""       </div>\n"""
        text += f"""    </li>"""
        carousel.append(text)

    carousel.extend([
        """  </ol>""",
        """  <aside class="carousel__navigation">""",
        """    <ol class="carousel__navigation-list">"""])

    for k in range(1, npub + 1):
        text  = f"""      <li class="carousel__navigation-item">\n"""
        text += f"""        <a href="#carousel__slide{k}" class="carousel__navigation-button">Go to {k}</a>\n"""
        text += f"""      </li>"""
        carousel.append(text)
    carousel.extend(["""    </ol>""", """  </aside>""", """</section>"""])

    return '\n'.join(carousel)

In [12]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}_content"' for k in range(1, npub + 1)])

script = f"""
const docs = [{docs}]

const slides = [{slides}]
""" + """
async function run() {
    for (let i = 0; i < docs.length; i++) {
        let file = await fetch(docs[i]);
        let text = await file.text()
        document.getElementById(slides[i]).innerHTML =
            marked.parse(text);
    }
    hljs.highlightAll();
}
run()
"""

page = f"""<!doctype html>
<html lang="en">

<head>
  <meta charset="utf-8">
  <meta name="viewport" content="width=device-width, initial-scale=1">
  <!-- Bootstrap CSS -->
  <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.0.2/dist/css/bootstrap.min.css" rel="stylesheet"
   integrity="sha384-EVSTQN3/azprG1Anm3QDgpJLIm9Nao0Yz1ztcQTwFspd3yD65VohhpuuCOmLASjC" crossorigin="anonymous">
  <!-- highlight.js CSS -->
  <link rel="stylesheet" href="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.1.0/styles/default.min.css">
  <!-- Mathjax 3 -->
  <script type="text/javascript" id="MathJax-config" src="mathjax_config.js"> </script>
  <script type="text/javascript" id="MathJax-script" async 
    src="https://cdn.jsdelivr.net/npm/mathjax@3/es5/tex-mml-chtml.js">
  </script>
  <link rel="stylesheet" href="index_carousel.css">
  <link rel="icon" type="image/x-icon" href="https://www.mpia.de/assets/touch-icon-32x32-a66937bcebc4e8894ebff1f41a366c7c7220fd97a38869ee0f2db65a9f59b6c1.png">
  <title>MPIA Arxiv on deck!</title>
</head>

<body>
  <div id="header"> <img src="header_banner.png" width="100%"></div>
  <div id="suptitle"> 7-day archives </div>
  <div id="info">
    <img src="https://pngimg.com/uploads/github/github_PNG58.png" height=30rem></img>
    <a href=https://github.com/mpi-astronomy/arxiv_display style="color:black;">github/mpi-astronomy/arxiv_display</a> 
  </div>
  {carousel:s}
</body>

<!-- Render Markdown -->

<body>
  <!-- highlight.js: https://highlightjs.org/download/ -->
  <script src="https://cdnjs.cloudflare.com/ajax/libs/highlight.js/11.5.0/highlight.min.js"></script>
  <!-- marked.js -->
  <script src="https://cdn.jsdelivr.net/npm/marked/marked.min.js"></script>
  <script>{script:s}</script>
</body>
</html>
"""
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

# Debugging papers